<a href="https://colab.research.google.com/github/Tanay-27/ML_Hackathons-Projects/blob/master/HackerEarth_Snake_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset.zip to dataset.zip


In [1]:
!ls

data  dataset.zip  model_baseline.h5  sample_data


In [2]:
# Importing Necessary Libraries
import numpy as np
import pandas as pd
import shutil
import cv2
import os

In [4]:
# The data needs to be preprocessed, we need to make new folders for each species and 
# Shift the corresponding image to the respcetive folder
#dataset = pd.read_csv('dataset/train.csv')
#file_names = list(dataset['image_id'].values)
#img_labels = list(dataset['breed'].values)

In [ ]:
lst = list(np.unique(dataset['breed'].values))

In [ ]:
source = os.getcwd()

for new_path in lst:
    if not os.path.exists("." + new_path):
        os.makedirs(new_path)

In [ ]:
folders = lst.copy()

for f in range(len(file_names)):
    current_img = file_names[f]
    current_label = img_labels[f]
    shutil.move("dataset//train//"+current_img+".jpg", current_label+"//"+current_img+".jpg")

In [ ]:
for label in os.listdir("data"):
    for image in os.listdir("data//"+label):
        img = cv2.resize(cv2.imread("data//"+label+"//"+image),(128,128))
        cv2.imwrite("data//"+label+"//"+image,img)

In [ ]:
# From here we start building the image classifier.
# The preprocessing will allow us to use data flow from directry function so as to reduce time consumption 
# And enable data augmentation
# Each image is resized to fit 128x128 and retained all the 3 layers for each image

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

In [15]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        featurewise_center= False,
        featurewise_std_normalization=False,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        vertical_flip=True,
        horizontal_flip=True,
        zca_epsilon=1e-06,
        brightness_range = (0.1,0.3))

In [16]:
gen = train_datagen.flow_from_directory(
        'data',
        target_size=(128, 128),
        batch_size=128,
        class_mode='categorical',
        color_mode="rgb",
        shuffle=True,
    seed=None,)

Found 5508 images belonging to 35 classes.


In [17]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import  Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline

In [28]:
# define the model and start building the cnn
model = Sequential()

In [29]:
from keras.layers import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

In [30]:
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (128,128,3)))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size = (5,5), padding='Same' , activation='relu' , input_shape = (128,128,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [31]:
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same',activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64 , kernel_size = (5,5), padding='Same' , activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))
model.add(Dropout(0.2))

In [32]:
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(35, activation = "softmax"))

In [33]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss = 'categorical_crossentropy', 
                optimizer = optimizer, 
                metrics = ['accuracy'])

In [ ]:
start = time.time()
model.fit_generator(
    gen,
    steps_per_epoch=50,
    epochs=20)
end = time.time()
print('Processing time:',(end - start)/60)
model.save_weights('model_baseline.h5')

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/20
 2/50 [>.............................] - ETA: 21:01 - loss: 50.0338 - accuracy: 0.0273